<a href="https://colab.research.google.com/github/YashShekhar/Generative-AI/blob/main/1_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain
Model, Promt, Prompt Templates and Chain

In [4]:
!pip install -qU "langchain[groq]"

In [5]:
from google.colab import userdata
apiKey = userdata.get('GROQ_API_KEY')

In [9]:
from langchain.chat_models import init_chat_model
model = init_chat_model(
    model="llama-3.3-70b-versatile",
    model_provider="groq",
    api_key=apiKey
)

In [18]:
response = model.invoke("Suggest me a title of movie in hindi to watch")
print(response.content)

एक अच्छी हिंदी फिल्म का नाम यह है: 

"दिलवाले दुल्हनिया ले जाएंगे" (Dilwale Dulhania Le Jayenge)

यह फिल्म एक रोमांटिक कॉमेडी है, जिसमें शाहरुख खान और काजोल ने मुख्य भूमिकाएँ निभाई हैं। यह फिल्म आपको हंसाएगी, रुलाएगी और आपको प्यार की सच्ची भावना को समझने में मदद करेगी।

क्या आप इस फिल्म को देखना चाहते हैं?


In [20]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English to Italian"),
    HumanMessage("hi")
]

response = model.invoke(messages)
print(response.content)

Ciao!


In [21]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from english to {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", "{text}")
    ]
)

In [22]:
prompt = prompt_template.invoke(
    {
        "language": "hindi",
        "text": "what is you age"
    }
)

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from english to hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is you age', additional_kwargs={}, response_metadata={})])

In [23]:
model.invoke(prompt)

AIMessage(content='आपकी उम्र क्या है?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 46, 'total_tokens': 57, 'completion_time': 0.04, 'prompt_time': 0.002315293, 'queue_time': 0.200313195, 'total_time': 0.042315293}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_90c1d253ff', 'finish_reason': 'stop', 'logprobs': None}, id='run-4025aaa9-6892-4892-a2ce-a51fd7867af1-0', usage_metadata={'input_tokens': 46, 'output_tokens': 11, 'total_tokens': 57})

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt = prompt_template.invoke({"topic": "cats"})

response = model.invoke(prompt)

response.content

'Why did the cat join a band? Because it wanted to be a purr-cussionist.'

In [43]:
movie_title_template = PromptTemplate.from_template("Suggest me a title of language {language} and of genre {genre} make sure to give only one suggestion")

movie_title_prompt = movie_title_template.invoke({
    "language": "hindi",
    "genre": "comedy"
})

response = model.invoke(movie_title_prompt)

print(response.content)

"Golmaal"


In [44]:
from langchain_core.output_parsers import StrOutputParser

movie_title_chain = movie_title_template | model | StrOutputParser()

response = movie_title_chain.invoke({"language": "english", "genre":"horror"})

print(response)

"The Shattered Reflection"


In [45]:
movie_summary_prompt = PromptTemplate.from_template("Give me 2-3 line summary of the movie {movie_title}")

In [50]:
from langchain_core.runnables import RunnableLambda
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

print_title_step = RunnableLambda(lambda x : print(x["movie_title"]))

composed_chain = {"movie_title": movie_title_chain} | print_title_step | movie_summary_prompt | model | StrOutputParser()

response = composed_chain.invoke({"language": "english", "genre":"horror"}, config={"callbacks":[StreamingStdOutCallbackHandler()]})

response

"The Shining"


'There is no movie titled "None". It seems you may have provided an incorrect or non-existent title. If you could provide a correct title, I\'d be happy to give you a summary of the movie.'